In [1]:
import os
for env in ["MKL_NUM_THREADS", "NUMEXPR_NUM_THREADS", "OMP_NUM_THREADS"]:
    os.environ[env] = "1"

import numpy as np
from numpy.random import default_rng
import numpy.random as rnd
from ttictoc import tic, toc

import approxbayescomp as abc

In [2]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)

tic()

ABC version: 0.1.0
Python version: 3.8.10 | packaged by conda-forge | (default, May 11 2021, 06:25:29) 
[GCC 9.3.0]
Numpy version: 1.21.0


In [3]:
numIters = numItersData = 10
popSize = 1000
smcArgs = {"verbose": True, "numProcs": 64}

## Inference of a Negative Binomial - Weibull model

In this notebook we are are conducting a simulation experiment where the claim frequency are Negative Binomial distributed 

$$
n_s\underset{\textbf{i.i.d.}}{\sim}\text{Neg-Bin}(\alpha = 4, p = 2/3),\text{ }s = 1,\ldots, 30
$$ 

and the individual claim sizes are weibull distributed

$$
u_1,\ldots, u_{n_s}\underset{\textbf{i.i.d.}}{\sim}\text{Weib}(k = 1/2, \beta = 1),\text{ }s = 1,\ldots 30.
$$ 

The available data is aggregated claim sizes in excess of the priority $c=1$ asociated to aa global stop-loss treaty, we have 

$$
x_s = \left(\sum_{k = 1}^{n_s}u_k-c\right)_{+},\text{ }s = 1,\ldots, t.
$$

Our aim is to look into the finite sample performance of our ABC implementation when the model is well specified that is when we assume a negative binomial - weibull model

In [4]:
rg = default_rng(123)

sample_sizes = [50, 250]
T = sample_sizes[-1]
t = np.arange(1, T + 1, 1)

# Frequency-Loss Model
α, p, k, β = 4, 2 / 3, 1 / 3, 1
θ_True = α, p, k, β
θ_sev = k, β
θ_freq = α, p
freq = "negative binomial"
sev = "weibull"

# Aggregation process
c = 0.25
psi = abc.Psi("GSL", c)

freqs, sevs = abc.simulate_claim_data(rg, T, freq, sev, θ_True)
xData = abc.compute_psi(freqs, sevs, psi)

In [5]:
[np.sum(xData[:ss] > 0) for ss in sample_sizes]

[30, 155]

In [6]:
params = ("α", "p", "r", "m")
prior = abc.IndependentUniformPrior([(0, 10), (1e-3, 1), (0, 10), (0, 50)], params)
model = abc.Model("negative binomial", "gamma", psi, prior)

In [7]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    %time fit = abc.smc(numIters, popSize, xDataSS, model, **smcArgs)

Final population dists <= 9.32, ESS = 649
CPU times: user 19.4 s, sys: 1.22 s, total: 20.6 s
Wall time: 30.9 s


Final population dists <= 6.02, ESS = 776
CPU times: user 12.9 s, sys: 459 ms, total: 13.3 s
Wall time: 31.7 s


In [8]:
params = ("r", "m")
prior = abc.IndependentUniformPrior([(0, 10), (0, 50)], params)

In [9]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    nData = freqs[:ss]

    model = abc.Model(nData, "gamma", psi, prior)

    %time fit = abc.smc(numItersData, popSize, xDataSS, model, **smcArgs)

Final population dists <= 7.13, ESS = 893
CPU times: user 19 s, sys: 978 ms, total: 20 s
Wall time: 40.3 s


Final population dists <= 4.97, ESS = 878
CPU times: user 18.5 s, sys: 934 ms, total: 19.5 s
Wall time: 1min 32s


In [10]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 197 secs = 3.28 mins
